<a href="https://colab.research.google.com/github/LC1332/Mini-Luotuo/blob/main/3.5B_minimal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 迷你骆驼:一系列蒸馏指令数据得到的中文语言模型

[![Code License](https://img.shields.io/badge/Code%20License-Apache_2.0-green.svg)]()
[![Data License](https://img.shields.io/badge/Data%20License-CC%20By%20NC%204.0-red.svg)]()

<p align="center"> <a href="https://scholar.google.com/citations?user=imroB-8AAAAJ&hl=zh-CN" target="_blank">黄钟健 @ 西安电子科大</a>, <a href="https://github.com/LC1332" target="_blank">李鲁鲁 @ 商汤科技</a></p>


<details>
  <summary> 黄钟健 训练了本项目的第一个模型 </summary>

李鲁鲁发起了项目，并提出了后续使用feature进行蒸馏的思路。

黄钟健 训练了本项目的第一个模型，并且实现了训练框架

后续如果更多其他的同学训练小模型，我们也会陆续加入到作者列表中

</details>

项目地址https://github.com/LC1332/Mini-Luotuo

本notebook由黄钟健实现

目前包含3.5B模型的inference代码

安装必要的环境

In [ ]:
!pip install bitsandbytes datasets sentencepiece transformers accelerate

In [2]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch

def generate_prompt(instruction, input=None):
    if input:
        return f"Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:\n"
    else:
        return f"Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Response:\n"


In [3]:
tokenizer = GPT2Tokenizer.from_pretrained('Midkey/GPT2-3.5B-chinese-ft-luotuo')

model = GPT2LMHeadModel.from_pretrained('Midkey/GPT2-3.5B-chinese-ft-luotuo', trust_remote_code=True, load_in_8bit=True, device_map='auto')

In [4]:
def evaluate(
    instruction,
    input=None,
    max_new_tokens=256,
):
    prompt = generate_prompt(instruction, input)
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].cuda()
    with torch.no_grad():
        generation_output = model.generate(
            input_ids=input_ids,
            return_dict_in_generate=True,
            output_scores=True,
            max_new_tokens=max_new_tokens,
            pad_token_id=50256
        )
    for s in generation_output.sequences:
        output = tokenizer.decode(s)
        print("Response:", output.split("### Response:")[1].strip())

现在可以玩了

In [9]:
instruction = '华中师范大学在什么地方?'
evaluate(instruction)

Response: 华中师范大学在湖北省武汉市洪山区<|endoftext|>


In [6]:
instruction = '蛋糕和巧克力做法上有什么区别？'
evaluate(instruction)

Response: 蛋糕和巧克力的做法上有很大的不同。蛋糕是用黄油，糖，面粉，可可粉，鸡蛋和牛奶等食材制作而成的，口感细腻，味道浓郁。而巧克力是用可可豆，坚果，奶油，糖，香草精，巧克力酱，黄油等食材制作而成的，口感比较柔软，味道比较甜。

蛋�


Response: 蛋糕和巧克力的做法上有很大的不同。蛋糕是用黄油，糖，面粉，可可粉，鸡蛋和牛奶等食材制作而成的，口感细腻，味道浓郁。而巧克力是用可可豆，坚果，奶油，糖，香草精，巧克力酱，黄油等食材制作而成的，口感比较柔软，味道比较甜。

蛋�

In [8]:
instruction = '新手司机开车上路要注意什么?'
evaluate(instruction)

Response: 新手司机开车上路要注意什么?

1. 注意车速，不要超速行驶。
2. 注意观察路况，遇有交通拥堵或事故，要提前减速慢行。
3. 注意观察车辆周围交通环境，避免车辆长时间停放影响其他车辆通行。
4. 注意车辆前后左右，避免车辆长时间停放影响其他�


In [ ]:
while 1:
    instruction = input('Ask something: ')
    evaluate(instruction)

TODO

- [ ] 一个gradio的游玩GUI
- [ ] 更好的蒸馏策略